# On crée le modèle

In [63]:
# Importe le modèle préparé par PE
from model.deepNN import DeepNetwork

# Première couche cachée: 10 neurones 
# Deuxième couche cachée: 5 neurones
# Entrée: 1 action 
# Sortie: 2 probabilités de chaque outcome
model_ML = DeepNetwork(hidden_size=[10, 5], input_size=1, output_size=2)
model_ML.to("cpu")

liste hidden init [10, 5]


DeepNetwork(
  (fc1): Linear(in_features=1, out_features=10, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=10, out_features=5, bias=True)
  )
  (fc4): Linear(in_features=5, out_features=2, bias=True)
)

# On tokenize le vocabulaire

In [64]:
# Import le tokenizer préparé par PE
from model.Tokenizer import SimpleTokenizerV1
from outil import create_dico_numerate_word

# Notre vocabulaire:
all_word = create_dico_numerate_word(['x', 'y', 'a', 'b'])
# Voici all_word : {'x': 0, 'y': 1, 'a': 2, 'b': 3}
print("Vocabulaire:", all_word)
# Nous n'avons pas besoin d'un tokenzier complexe, 
# il va avoir le même rôle qu'un dictionnaire
tokenizer = SimpleTokenizerV1(all_word)
# Maintenant pour encoder une action ou un outcome nous pouvons faire :
print('Test encode a:', tokenizer.encode('a'))
# et pour decoder :
print('Test decode l:', tokenizer.decode(1))

Vocabulaire: {'x': 0, 'y': 1, 'a': 2, 'b': 3}
Test encode a: 2
Test decode l: y


# On définit les paramètres du modèle

Le `lr` et le `weight_decay` sont des paramètres très importants qui vont permettre de corriger plus ou moins vite le modèle. Une correction rapide est moins précise.


In [65]:
lr = 1e-1
weight_decay = 1e-2

In [66]:
# On importe les outils fournis par torch
import torch
import torch.nn as nn

# Le modèle a besoin d'un optimizer pour corriger les poids.
optimizer = torch.optim.Adam(model_ML.parameters(), lr=lr, weight_decay=weight_decay)

# Nous avons aussi besoin d'une fonction de loss, par exemple la CrossEntropy
# Au vue de la simplicité de prédiction le type de loss importe peu
loss_func = nn.CrossEntropyLoss()

# On crée un dataset

l'action `a` renvoie l'outcome `x`. L'action `b` renvoie l'outcome `y`.

In [121]:
tokenized_actions = [[tokenizer.encode('a')], [tokenizer.encode('b')]]
tokenized_outomes = [tokenizer.encode('x'), tokenizer.encode('y')]

# Convertit en tensors pour torch
x = torch.tensor(tokenized_actions, dtype=torch.float).to("cpu")
y = torch.tensor(tokenized_outomes, dtype=torch.long).to("cpu")

# Notre dataset de test :
print("x (tokenized actions):\n", x)
print("y (tokenized outcomes):\n", y)

x (tokenized actions):
 tensor([[2.],
        [3.]])
y (tokenized outcomes):
 tensor([0, 1])


On crée un loader pour charger le dataset dans le modèle

In [32]:
# Nous utilisons un dataLoader de pytorch
# Le batch_size n'a pas d'importance de même que le shuffle
validate_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(x, y),
    batch_size=32, shuffle=False
)

# La fonction fit()

In [68]:
def fit(action, outcome):
    action = tokenizer.encode(action)
    action = torch.tensor([action], dtype=torch.float).to("cpu")
    outcome = tokenizer.encode(outcome)
    outcome = torch.tensor(outcome, dtype=torch.long).to("cpu")
    outcome = torch.nn.functional.one_hot(outcome, num_classes=2).to(torch.float)
    # Qu'est ce que ca fait ?
    model_ML.train()

    train(model=model_ML, 
          train_data=[(action, outcome)],
          optimizer=optimizer,
          loss_func=loss_func,
          nb_epochs=1,
          validate_loader=validate_loader,
          print_=True)

In [170]:
fit('a', 'x')
fit('b', 'y')

Epoch 1/1, 0.5 , Loss: 0.6984
Epoch 1/1, 0.5 , Loss: 0.6932


# La fonction predict()

In [172]:
def predict(action):
    action = tokenizer.encode(action)
    action = torch.tensor([action], dtype=torch.float).to("cpu")
    model_ML.eval() # On peut le passer en mode evaluation
    x = model_ML(action)
    print(x)
    x = torch.argmax(x, dim=0).item()
    return tokenizer.decode(x)

In [174]:
predicted_outcome = predict('a')
print("Predication pour a:", predicted_outcome)
predicted_outcome = predict('b')
print("Predication pour b:", predicted_outcome)

tensor([0.2534, 0.2635], grad_fn=<ViewBackward0>)
Predication pour a: y
tensor([0.2534, 0.2635], grad_fn=<ViewBackward0>)
Predication pour b: y
